In [ ]:
import torch
import torch.nn as nn # 다양한 Neuron Network들을 사용하기 위해서 사용.
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader # 학습시 batchszie 단위로 학습하며, 데이터 관리의 편리성을 위해서 사용. 본 코드에서는 사용되지 않음.

# 딕셔너리 value로 key 값 찾기
def get_key(dic, val):
    for key, value in dic.items():
         if val == value:
             return key
sequence = 'helloworld' # 문장 정의.
set_sequence = list(set(sequence)) # 겹치는 애들 제거
dic = {c: i for i, c in enumerate(set_sequence)} # 딕셔너리 comprehension 이용해서 딕셔너리 선언 value : index 순서로

print(f'sequence : {sequence}, set_sequence : {set_sequence}')
x_data = [dic[c] for c in sequence[:-1]] # helloworl
y_data = [dic[c] for c in sequence[1:]] # elloworld
print('x_data = ',sequence[:-1],  x_data)
print('y_data = ',sequence[1:] , y_data)
# one-hot encoding & tensor
x_one_hot = [np.eye(len(dic))[x] for x in x_data]
X = torch.FloatTensor(x_one_hot) # batch size x
Y = torch.LongTensor(y_data)
print(X.shape, Y.shape)
X = torch.FloatTensor([x_one_hot]) # batch size o
Y = torch.LongTensor([y_data])
print(X.shape, Y.shape)


# Model 설계
learning_rate = 0.01 # 학습률 정하기
training_epochs = 100 # 학습 횟수 정하기

# define cost/loss & optimizer
# input이 N x S x E
input_size = len(dic)
hidden_size = len(dic)
model = nn.RNN(input_size, hidden_size, batch_first=True) # 위에서 batch가 가장 먼저 나왓으므로/는 true이므로
criterion = nn.CrossEntropyLoss()    # Softmax
optimizer = optim.AdamW(model.parameters(), lr=learning_rate) # Adam 활용

# train
for epoch in range(training_epochs):
  optimizer.zero_grad() 
  outputs, _status = model(X)
  #print(outputs.reshape(-1, len(dic)).shape, Y.reshape(-1).shape)
  loss = criterion(outputs.reshape(-1, len(dic)), Y.reshape(-1)) 
  loss.backward()
  optimizer.step()
  result = outputs.data.numpy().argmax(axis=2)
  result_str = ''.join([set_sequence[c] for c in np.squeeze(result)]) # 학습을 확인하기 위해서 출력.
  acurracy = 0
  result_list = [dic[c] for c in result_str] # elloworld
  for i, v in enumerate(y_data):
    if result_list[i] == v:
      acurracy += 1
  print('epoch: ',epoch, 'loss: ', loss.item(), 'accuracy: ', round(acurracy/len(y_data), 3),'%')
  print('prediction: ', result_str, 'true Y: ', sequence[1:])

sequence : helloworld, set_sequence : ['l', 'r', 'd', 'o', 'h', 'e', 'w']
x_data =  helloworl [4, 5, 0, 0, 3, 6, 3, 1, 0]
y_data =  elloworld [5, 0, 0, 3, 6, 3, 1, 0, 2]
torch.Size([9, 7]) torch.Size([9])
torch.Size([1, 9, 7]) torch.Size([1, 9])
epoch:  0 loss:  2.0755155086517334 accuracy:  0.111 %
prediction:  wedrrhwhd true Y:  elloworld
epoch:  1 loss:  2.037285089492798 accuracy:  0.111 %
prediction:  wedrrewhd true Y:  elloworld
epoch:  2 loss:  2.0006048679351807 accuracy:  0.111 %
prediction:  lerrrewhd true Y:  elloworld
epoch:  3 loss:  1.9651970863342285 accuracy:  0.222 %
prediction:  lerrreeld true Y:  elloworld
epoch:  4 loss:  1.9310005903244019 accuracy:  0.333 %
prediction:  lelrreeld true Y:  elloworld
epoch:  5 loss:  1.8981225490570068 accuracy:  0.333 %
prediction:  lellreeld true Y:  elloworld
epoch:  6 loss:  1.8666930198669434 accuracy:  0.333 %
prediction:  lelllelld true Y:  elloworld
epoch:  7 loss:  1.8368418216705322 accuracy:  0.222 %
prediction:  lelllell

In [ ]:
import torch
import torch.nn as nn # 다양한 Neuron Network들을 사용하기 위해서 사용.
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader # 학습시 batchszie 단위로 학습하며, 데이터 관리의 편리성을 위해서 사용. 본 코드에서는 사용되지 않음.


# 딕셔너리 value로 key 값 찾기
def get_key(dic, val):
    for key, value in dic.items():
         if val == value:
             return key
sequence = 'helloworld' # 문장 정의.
set_sequence = list(set(sequence)) # 겹치는 애들 제거
dic = {c: i for i, c in enumerate(set_sequence)} # 딕셔너리 comprehension 이용해서 딕셔너리 선언 value : index 순서로
# Parameters
dic_size = len(dic)
input_size = dic_size
hidden_size = dic_size * 2
output_size = dic_size 
unit_sequence_length = 5 # hello, world

# dataset 분리 및 추가
input_batch = []
target_batch = []
for i in range(round(len(sequence)/ unit_sequence_length)):
  x_data = [dic[c] for c in sequence[i*5:(i+1)*5-1]]
  print('x_data ', x_data )
  x_one_hot = [np.eye(dic_size)[x] for x in x_data]
  y_data = [dic[c] for c in sequence[5*i+1:(i+1)*5]]
  print('y_data ', y_data )
  input_batch.append(x_one_hot)
  target_batch.append(y_data)

# To torch tensors
X = torch.FloatTensor(np.array(input_batch))
Y = torch.LongTensor(np.array(target_batch))
print(X.shape)
print(Y.shape)

class Custom_GRU(torch.nn.Module): # 기존에 GRU 모델이 있으므로 새로운 GRU 사용
  def __init__(self, input_dim, hidden_dim, output_dim, layers):
    super(Custom_GRU, self).__init__()
    self.rnn = torch.nn.GRU(input_dim, hidden_dim, num_layers=layers) # RNN/LSTM/GRU만 바꿔서 사용가능.
    self.fc = torch.nn.Linear(hidden_dim, output_dim, bias=True) 

  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

# Model 설계
learning_rate = 0.03 # 학습률 정하기
training_epochs = 300 # 학습 횟수 정하기
model2 = Custom_GRU(input_size, hidden_size, output_size, 2)
criterion = nn.CrossEntropyLoss()    # Softmax
optimizer = optim.AdamW(model2.parameters(), lr=learning_rate) # Adam 활용

# train
for epoch in range(training_epochs):
  optimizer.zero_grad() 
  outputs = model2(X)
  loss = criterion(outputs.reshape(-1, dic_size), Y.reshape(-1))
  loss.backward()
  optimizer.step()
  result = outputs.data.numpy().argmax(axis=2)
  # 결과 확인
  t_sen = ''
  a_sen = ''
  for index, sentence in enumerate(result):
    t_sen += ''.join([set_sequence[c] for c in np.squeeze(sentence)]) + ' '
    a_sen +=  sequence[5*index+1:5*index+5] + ' '
  acurracy = 0
  for i, v in enumerate(a_sen):
    if t_sen[i] == v:
      acurracy += 1
  print('epoch: ',epoch, 'loss: ', loss.item(), 'accuracy: ', round((acurracy-1)/(len(a_sen)-1), 3),'%')
  print('prediction:', t_sen, 'true Y:', a_sen)
outputs = model2(X)
result = outputs.data.numpy().argmax(axis=2)
print(result, result.shape)

x_data  [4, 5, 0, 0]
y_data  [5, 0, 0, 3]
x_data  [6, 3, 1, 0]
y_data  [3, 1, 0, 2]
torch.Size([2, 4, 7])
torch.Size([2, 4])
epoch:  0 loss:  2.004669189453125 accuracy:  0.111 %
prediction: wwww wwww  true Y: ello orld 
epoch:  1 loss:  1.9068130254745483 accuracy:  0.333 %
prediction: ewww llll  true Y: ello orld 
epoch:  2 loss:  1.815883994102478 accuracy:  0.444 %
prediction: llll llll  true Y: ello orld 
epoch:  3 loss:  1.7159028053283691 accuracy:  0.444 %
prediction: llll llll  true Y: ello orld 
epoch:  4 loss:  1.6136548519134521 accuracy:  0.444 %
prediction: llll llll  true Y: ello orld 
epoch:  5 loss:  1.528850793838501 accuracy:  0.444 %
prediction: llll llll  true Y: ello orld 
epoch:  6 loss:  1.4584949016571045 accuracy:  0.444 %
prediction: llll llll  true Y: ello orld 
epoch:  7 loss:  1.3755651712417603 accuracy:  0.444 %
prediction: llll llll  true Y: ello orld 
epoch:  8 loss:  1.2738069295883179 accuracy:  0.556 %
prediction: olll olll  true Y: ello orld 
epoch